# Initialization

In [ ]:
# Google Earth Engine packages
import ee
import geemap
import geemap.colormaps as cm

import xarray as xr
import numpy as np


# function to load nc file into GEE
def netcdf_to_ee(filename):
    ds = xr.open_dataset(filename)
    data = ds['z']

    lon_data = np.round(data['lon'], 3)
    lat_data = np.round(data['lat'], 3)

    dim_lon = np.unique(np.ediff1d(lon_data).round(3))
    dim_lat = np.unique(np.ediff1d(lat_data).round(3))

    if (len(dim_lon) != 1) or (len(dim_lat) != 1):
        print("The netCDF file is not a regular longitude/latitude grid")

    data_np = np.array(data)
    data_np = np.transpose(data_np)

    # Figure out if we need to roll the data or not
    # (see https://github.com/giswqs/geemap/issues/285#issuecomment-791385176)
    if np.max(lon_data) > 180:
        data_np = np.roll(data_np, 180, axis=0)
        west_lon = lon_data[0] - 180
    else:
        west_lon = lon_data[0]

    transform = [dim_lon[0], 0, float(west_lon) - dim_lon[0]/2, 0, dim_lat[0], float(lat_data[0]) - dim_lat[0]/2]

    image = geemap.numpy_to_ee(
        data_np, "EPSG:4326", transform=transform, band_names='z'
    )
    return image, data_np, transform

In [ ]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()
    
Map = geemap.Map()

In [ ]:
import configparser
import ast

# read local config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

# get file config from config.ini
dir_input = config['FILE_SETTINGS']['DIR_INPUT']
dir_output = config['FILE_SETTINGS']['DIR_OUTPUT']
output_gpkg = dir_output + config['FILE_SETTINGS']['GPKG_NAME']

# get date range for forcing data
date_range = ast.literal_eval(config['CONFIG']['DATE_RANGE'])

***

# ERA5L Geopotential

### Load input data

Load ERA5L file for geopotential and add to Map

In [ ]:
file = 'ERA5_land_Z_geopotential_HMA.nc'
filename = dir_input + file

In [ ]:
image, data_np, transform = netcdf_to_ee(filename)

In [ ]:
# add image as layer
vis_params =  {'min': int(data_np.min()), 'max': int(data_np.max()), 'palette': cm.palettes.terrain, 'opacity': 0.8}
Map.addLayer(image, vis_params, "ERA5L geopotential")

Load catchment and add to map

In [ ]:
import geopandas as gpd

catchment_new = gpd.read_file(output_gpkg, layer='catchment_new')

catchment = geemap.geopandas_to_ee(catchment_new)
Map.addLayer(catchment, {'color': 'darkgrey'}, "Catchment")
Map.centerObject(catchment, zoom=9)

Show Map

In [ ]:
Map

### Calculate weighted avarage geopotential and covert to elevation

In [ ]:
# execute reducer
dict = image.reduceRegion(ee.Reducer.mean(),
                          geometry=catchment,
                          crs='EPSG:4326',
                          crsTransform=transform)

# get mean value and print
mean_val = dict.getInfo()['z']
print(f'geopotential mean: {mean_val:.2f}, elevation: {mean_val / 9.80665:.2f}')

***

### ERA5L Temperature and Precipitation Time Series

In [ ]:
collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_RAW')\
    .select('temperature_2m','total_precipitation_sum')\
    .filterDate(date_range[0], date_range[1])\
    .filterBounds(catchment)

In [ ]:
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), catchment)
    return image.set(dict)


withMean = collection.map(setProperty)

In [ ]:
import pandas as pd
import datetime

df = pd.DataFrame()
df['ts'] = withMean.aggregate_array('system:time_start').getInfo()
df['temp'] = withMean.aggregate_array('temperature_2m').getInfo()
df['temp_corr'] = df['temp'] + 4.68
df['temp_corr_c'] = df['temp'] - 273.15
df['prec'] = withMean.aggregate_array('total_precipitation_sum').getInfo()
df['prec'] = df['prec'] * 1000
df['dt'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))

In [ ]:
df.to_csv(dir_output + 'ERA5L.csv',header=True,index=False)

In [ ]:
display(df)